In [31]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import numpy as np

def ngram_probs(filename='raw_sentences.txt'):
    with open(filename,'r') as file:
        content_text = file.read()
    
    tokenizer = Tokenizer(lower=True, split=" ")
    tokenizer.fit_on_texts([content_text])
    vocab = tokenizer.word_index
    vocab_size = len(vocab)
    
    vocab_reverse = {}
    vocab_keys = list(vocab.keys())
    for vocab_word in vocab_keys:
        vocab_index = vocab[vocab_word]
        vocab_reverse[vocab_index] = vocab_keys[vocab_index-1]
            
    content_word_ids = tokenizer.texts_to_sequences([content_text])
    content_word_ids = content_word_ids[0]
    content_len = len(content_word_ids)
    bigram_probs = np.zeros([vocab_size, vocab_size])
    trigram_probs = np.zeros([vocab_size, vocab_size])
    
    for i in range(content_len):
        if i-1>=0:
            bigram_probs[content_word_ids[i]-1,content_word_ids[i-1]-1] += 1
            trigram_probs[content_word_ids[i]-1,content_word_ids[i-1]-1] += 1
        if i+1<content_len:
            bigram_probs[content_word_ids[i]-1,content_word_ids[i+1]-1] += 1
            trigram_probs[content_word_ids[i]-1,content_word_ids[i+1]-1] += 1
        if i-2>=0:
            trigram_probs[content_word_ids[i]-1,content_word_ids[i-2]-1] += 1
        if i+2<content_len:
            trigram_probs[content_word_ids[i]-1,content_word_ids[i+2]-1] += 1
    
    return bigram_probs, trigram_probs, vocab, vocab_reverse

cnt2, cnt3, vocab ,_ = ngram_probs('E:/Raw data/test/raw_sentences.txt')
print(cnt2[(vocab['we'], vocab['are'])])


51.0


In [33]:
def prob3(bigram, cnt2=cnt2, cnt3=cnt3):
    p12 = cnt2[bigram[0],bigram[1]] / cnt2[bigram[0],:].sum() + cnt2[bigram[0],bigram[1]] / cnt2[bigram[1],:].sum()
    p123 = cnt3[:,bigram[0]] / cnt3[bigram[0],:].sum() + cnt3[:,bigram[1]] / cnt3[bigram[1],:].sum()
    prob = p123/p12
    return prob

p = prob3([vocab['we'], vocab['are']])
print(p[vocab['family']])

0.5285529502007911


In [36]:
def predict_max(starting, cnt2=cnt2, cnt3=cnt3):
    list_of_words = starting
    word_next = ''
    while word_next=='.' or len(list_of_words)>=15:
        p_word = []
        n_word = []
        for word in vocab:
            p_word.append(p[word])
            n_word.append(word)
        p_max = np.array(p_word).argmax()
        word_next = n_word[p_max]
        list_of_words.append(word_next)
    
    return list_of_words

sent = predict_max(['we', 'are'])
assert sent[-1] == '.' or len(sent) <= 15
print(' '.join(sent))

we are
